In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learnig
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [ ]:
# Carregar dados
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
# União das duas tabelas para o tratamento da base
frame = pd.concat([train_df, test_df])

In [ ]:
frame.info()

In [ ]:
# A variável "Cabin" será excluída, pois possui muitos valores ausentes e não há possibilidade de gerar artificialmente
# novos valores.
# A variável "Ticket" também será excluída, pois são valores alfanuméricos sem padrão definido e não são passíveis de 
# ranqueamento
# A variável nome não interfere no resultado, também será excluída

frame.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

## Classficação das variáveis

In [ ]:
size = frame['Survived'].value_counts()
labels = ['Morreu', 'Viveu']
explode = [0.1, 0]

plt.pie(size, explode=explode, labels=labels, autopct='%1.1f', shadow=True, startangle=270)
plt.title('Percentual de sobreviventes')
plt.show()


In [ ]:
colunas = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
plt.figure(figsize=(16, 10))

for i in range(0, len(colunas)):
    df_plot = frame.groupby([colunas[i], 'Survived']).count()['PassengerId'].rename('count')
    df_plot = (df_plot / df_plot.groupby(level=0).sum()).reset_index()
    df_plot = df_plot[df_plot['Survived'] == 0]

    plt.subplot(2, 3, i+1)
    plt.bar(df_plot[colunas[i]], df_plot['count'])
    plt.title(colunas[i])
    plt.ylabel('Não sobreviveu')

plt.show()

In [ ]:
corr_matrix = frame.drop(['Survived', 'PassengerId'], axis=1)
corr_matrix = corr_matrix.corr().abs()
plt.figure(figsize=(15, 15))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
frame[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
frame[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
frame[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
frame[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
frame[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

# Transformação

In [ ]:
# Transformar Sex e Embarked em valor numérico
# Porto embarque: C = Cherbourg, Q = Queenstown, S = Southampton

sex = {'male': 0, 'female': 1}
frame['Sex'] = frame['Sex'].map(sex)

port = {'C': 1, 'Q': 2, 'S': 3}
frame['Embarked'] = frame['Embarked'].map(port)

In [ ]:
mean_ages = np.zeros((2, 3))

In [ ]:
# Completar
# 1 - as idades das pessoas que não possem a informação com a média para a classe e sexo

for i in range(2):
    for j in range(3):
        ages = frame.loc[(frame['Sex'] == i) & (frame['Pclass'] == j+1)]['Age'].mean()
        mean_ages[i, j] = ages

for i in range(2):
    for j in range(3):
        frame.loc[(frame['Age'].isnull()) & (frame['Sex'] == i) & (frame['Pclass'] == j+1), 'Age'] = mean_ages[i, j]

In [ ]:
# 2 - o local de embarque para duas pessoas que não apresentam essa informação
# Será utilizada a moda dos valores de embarque.

frame['Embarked'].fillna(frame['Embarked'].dropna().mode()[0], inplace=True)

In [ ]:
# 3 - o Fare para uma pessoa não apresenta valor
# Será utilizada a média dos valores considerando local de embarque e a classe.

frame['Fare'].fillna(frame[(frame['Fare'].notnull()) & (frame['Pclass'] == 3) & (frame['Embarked'] == 3)]['Fare'].mean(),
                         inplace=True)

In [ ]:
# Normalização/padronização das informações de Age e de Fare
# StandardScaler
scaler = StandardScaler()

In [ ]:
features = ['Age', 'Fare']

for i in features:
    var = np.array(frame[i]).reshape(-1, 1)
    scaler.fit(var)
    frame[i] = scaler.transform(var)
    plt.hist(frame[i])
    plt.show()

In [ ]:
df_treino = frame[frame['Survived'].notnull()]
df_teste = frame[frame['Survived'].isnull()]

In [ ]:
# Split do DataFrame de treino em duas partes, para validar os modelos

Y = df_treino['Survived']
X = df_treino.drop(['PassengerId', 'Survived'], axis=1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

print('X_train: ', X_train.shape)
print('X_test: ', X_test.shape)

# Randon Forest

In [ ]:
# Primeiro avaliar os resultados do treino, utilizando as métricas para identificar overfitting
# Depois fazer a previsão.
# Técnicas:
# 1-Cross Validation
# 2-ROC

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, Y_train)

# Fazendo previsão
Y_pred = rf.predict(X_test)


In [ ]:
# Cross validation do treino
# Se acurácia das bases de treino e de teste forem próximas, pode indicar que não há overfitting
# Usar a base de treino para fazer a 1ª validação

scores_rf = cross_val_score(rf, X_train, Y_train, cv=10, scoring='accuracy', n_jobs= -1)
print('Cross validation treino: Média = {:.3f} Desvio Padrão = {:.3f}' .format(np.mean(scores_rf), np.std(scores_rf)))

print('----------------------')

#Confusion Matrix
print('Confusion Matrix em Teste')
print(confusion_matrix(Y_test, Y_pred))

print('----------------------')

# Acurácia das previsões em relação ao resultado real
print('Acurácia em Teste: ', accuracy_score(Y_test, Y_pred))

# Gradient Boosting Classifier

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0)
gbc.fit(X_train, Y_train)

# Previsão
Y_pred = gbc.predict(X_test)

In [ ]:
scores_gbc = cross_val_score(gbc, X_train, Y_train, cv=10, scoring='accuracy', n_jobs= -1)
print('Cross validation treino: Média = {:.3f} Desvio Padrão = {:.3f}' .format(np.mean(scores_gbc), np.std(scores_gbc)))

print('----------------------')

#Confusion Matrix
print('Confusion Matrix em Teste')
print(confusion_matrix(Y_test, Y_pred))

print('----------------------')

# Acurácia das previsões em relação ao resultado real
print('Acurácia em Teste: ', accuracy_score(Y_test, Y_pred))

# Regressão Logística

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, Y_train)

# Fazendo previsão
Y_pred = lr.predict(X_test)

In [ ]:
scores_lr = cross_val_score(lr, X_train, Y_train, cv=10, scoring='accuracy', n_jobs= -1)
print('Cross validation treino: Média = {:.3f} Desvio Padrão = {:.3f}' .format(np.mean(scores_lr), np.std(scores_lr)))

print('----------------------')

#Confusion Matrix
print('Confusion Matrix em Teste')
print(confusion_matrix(Y_test, Y_pred))

print('----------------------')

# Acurácia das previsões em relação ao resultado real
print('Acurácia em Teste: ', accuracy_score(Y_test, Y_pred))

# Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train, Y_train)

# Fazendo previsão
Y_pred = nb.predict(X_test)

In [ ]:
scores_nb = cross_val_score(nb, X_train, Y_train, cv=10, scoring='accuracy', n_jobs= -1)
print('Cross validation treino: Média = {:.3f} Desvio Padrão = {:.3f}' .format(np.mean(scores_nb), np.std(scores_nb)))

print('----------------------')

#Confusion Matrix
print('Confusion Matrix em Teste')
print(confusion_matrix(Y_test, Y_pred))

print('----------------------')

# Acurácia das previsões em relação ao resultado real
print('Acurácia em Teste: ', accuracy_score(Y_test, Y_pred))

# Stochastic Gradient Descent

In [ ]:
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)

# Fazendo previsão
Y_pred = sgd.predict(X_test)

In [ ]:
scores_sgd = cross_val_score(sgd, X_train, Y_train, cv=10, scoring='accuracy', n_jobs= -1)
print('Cross validation treino: Média = {:.3f} Desvio Padrão = {:.3f}' .format(np.mean(scores_sgd), np.std(scores_sgd)))

print('----------------------')

#Confusion Matrix
print('Confusion Matrix em Teste')
print(confusion_matrix(Y_test, Y_pred))

print('----------------------')

# Acurácia das previsões em relação ao resultado real
print('Acurácia em Teste: ', accuracy_score(Y_test, Y_pred))

# Support Vector Machine

In [ ]:
svm = SVC()
svm.fit(X_train, Y_train)

# Fazendo previsão
Y_pred = svm.predict(X_test)

In [ ]:
scores_svm = cross_val_score(svm, X_train, Y_train, cv=10, scoring='accuracy', n_jobs= -1)
print('Cross validation treino: Média = {:.3f} Desvio Padrão = {:.3f}' .format(np.mean(scores_svm), np.std(scores_svm)))

print('----------------------')

#Confusion Matrix
print('Confusion Matrix em Teste')
print(confusion_matrix(Y_test, Y_pred))

print('----------------------')

# Acurácia das previsões em relação ao resultado real
print('Acurácia em Teste: ', accuracy_score(Y_test, Y_pred))

In [ ]:
total_scores = pd.DataFrame({'Modelo': ['Random Forest', 'Gradient Boosting', 'Logistic Regression', 'Naive Bayes', 
                            'Stochastic Gradient', 'Support Vector Machine'], 
                            'Score_Cross_Validation': [np.mean(scores_rf), np.mean(scores_gbc), np.mean(scores_lr), 
                            np.mean(scores_nb), np.mean(scores_sgd), np.mean(scores_svm)]})

total_scores.sort_values(by='Score', ascending=False)

In [ ]:
# Aplicando modelo com melhor resultado no dataframe de teste
Y_pred = df_teste.drop(['PassengerId', 'Survived'], axis=1)
Y_pred = svm.predict(Y_pred)


In [ ]:
submission = pd.DataFrame({'PassengerId': df_teste['PassengerId'],
                        'Survived': Y_pred})

submission.to_csv('submission.csv', index=False)